In [35]:
%load_ext autoreload
%autoreload 2

from schema2db import validate
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np

In [12]:
inputfile = '../tests/testdata/testschema1.sql'
with open(inputfile, 'r') as f:
    inputstr = f.read()

In [14]:
components = inputstr.split(";")

In [15]:
components[0]

'create table prices (\n   itemid int(16) unsigned not null,\n   itemname varchar(20) not null,\n   price decimal(9,6) signed not null,\n   primary key (itemid)\n)'

In [16]:
table_str = components[0]

In [215]:
def clean_doc(doc):
    """clean out extra spaces
    """
    lines = doc.split('\n')
    lines = [" ".join(l.split()) for l in lines if l and not l.isspace()]
    return '\n'.join(lines)

def get_table_name(line):
    words = line.split()
    if words[1].lower() != 'table'.lower():
        raise ValueError("You can only create tables, not {}".format(words[1]))
    else:
        return words[2]

def extract_sql_doc(inputfile):
    """ Extract sql components from the entire document
    params:
    inputfile: str, name of the input sql file
    """
    with open(inputfile, 'r') as f:
        inputstr = f.read()
    blocks = clean_doc(inputstr).split(";\n")
    components = []
    for b in blocks:
        components.append(extract_sql_block(b))
    return components

def extract_sql_block(block):
    """ Parse a block of sql commands and return corresponding
    sql components
    """
    operation = block.split()[0].lower()
    if operation == 'create':
        return parse_create_block(block)
    elif operation == 'alter':
        return parse_alter_block(block)
    else:
        raise NotImplementedError("{} operation not supported".format(operation))

def extract_datatype(words):
    strtype = {}
    next_pos = 1
    strtype['type'] = (re.findall("[a-zA-Z]+", words[0]))[0].lower()
    strtype['args'] = re.findall("[0-9]+", words[0])
    if len(words) > 1 and words[1].lower() == 'signed':
        strtype['signed'] = True
        next_pos += 1
    elif len(words) > 1 and words[1].lower() == 'unsigned':
        strtype['signed'] = False
        next_pos += 1
    return strtype, words[next_pos:]

def extract_null(words):
    ifnull = None
    nextpos = 0
    if 'not null' in " ".join([s.lower() for s in words]):
        ifnull = False
        nextpos = 2
    elif 'null' in " ".join([s.lower() for s in words]):
        ifnull = True
        nextpos = 1
    if len(words) > nextpos:
        return ifnull, words[1:]
    else:
        return ifnull, None

def parse_create_block(sql_str):
    lines = sql_str.split('\n')
    table = {'columns': [], 'primary_keys': []}
    table['tablename'] = get_table_name(lines[0])
    for l in lines[1:]:
        if 'primary key' in l.lower():
            table['primary_keys'].append(parse_keys(l))
        elif re.findall("[a-zA-Z]+", l):
            table['columns'].append(parse_items(l))
    return table

def parse_items(l):
    c = l.split()
    cmds = []
    cmds = {'name': c[0]}
    cmds['type'], cmd_remaining = extract_datatype(c[1:])
    cmds['null'], cmd_remaining = extract_null(cmd_remaining)
    return cmds

def parse_keys(l):
    tokens = re.findall("[a-zA-Z]+", l)
    return [t for t in tokens if t.lower() not in('primary', 'key')]

def parse_alter_block(sql_cmds):
    constraints = ["constraints placeholder"]
    return constraints

In [216]:
extract_sql_doc('../tests/testdata/testschema1.sql')

[{'columns': [{'name': 'itemid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'itemname',
    'type': {'type': 'varchar', 'args': ['20']},
    'null': False},
   {'name': 'price',
    'type': {'type': 'decimal', 'args': ['9', '6'], 'signed': True},
    'null': False}],
  'primary_keys': [['itemid']],
  'tablename': 'prices'},
 {'columns': [{'name': 'saleid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'itemid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'saledate', 'type': {'type': 'date', 'args': []}, 'null': False},
   {'name': 'quantity',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'buyername',
    'type': {'type': 'varchar', 'args': ['60']},
    'null': True}],
  'primary_keys': [['saleid']],
  'tablename': 'sales'},
 ['constraints placeholder']]

In [208]:
re.findall("[a-zA-Z]+", "primary   key (one, two, three)")

['primary', 'key', 'one', 'two', 'three']

In [91]:
parse_create(table_words[3])

{'colname': 'price',
 'type': {'type': 'decimal', 'args': ['9', '6'], 'signed': True},
 'null': False}

In [50]:
" ".join(" not  null".split())

'not null'